# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [2]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [7]:
#Definition of the tables.
import pandas as pd

# Table and definitions sample
data = {
    'table': ['employees', 'salary', 'studies'],
    'definition': [
        'Employee information, name, position, etc.',
        'Salary details for each year',
        'Educational studies, institution name, type of studies, level'
    ]
}

df = pd.DataFrame(data)
print(df)


       table                                         definition
0  employees         Employee information, name, position, etc.
1     salary                       Salary details for each year
2    studies  Educational studies, institution name, type of...


In [8]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [9]:
print(text_tables)

employees: Employee information, name, position, etc.
salary: Salary details for each year
studies: Educational studies, institution name, type of studies, level


In [12]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Question:
{question}
"""

# Replace with actual question
user_question = "List employees who studied at a university."


In [13]:
# Creating the prompt
pqt1 = prompt_question_tables.format(tables=text_tables, question=user_question)

In [16]:
print(return_OAI(pqt1))

```json
{
    "tables": ["employees", "studies"]
}
```


In [21]:
pqt3 = prompt_question_tables.format(
    tables=text_tables,
    question="How many employees have a Master's degree?"
)


In [22]:
print(return_OAI(pqt3))

```json
{
    "tables": ["employees", "studies"]
}
```


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [28]:
# Import necessary libraries
import pandas as pd
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv

# Load OpenAI API Key from environment
load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Function to interact with OpenAI GPT model
def query_gpt(prompt):
    client = OpenAI(api_key=OPENAI_API_KEY)
    context = [{"role": "system", "content": prompt}]

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=context,
        temperature=0
    )

    return response.choices[0].message.content

# Define table descriptions using pandas DataFrame
data = {
    'table': ['employees', 'salary', 'studies'],
    'definition': [
        'Information about employees, including names, roles, and departments.',
        'Annual salary details for each employee.',
        'Educational background, institutions attended, degree types.'
    ]
}

df = pd.DataFrame(data)

# Convert table information into formatted string
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for _, row in df.iterrows()])

# Create a prompt with table definitions and user's scenario
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names clearly.
###User Question:
{question}
"""

# Example Scenario (Version 1)
scenario1 = "List all employees who earned more than $100,000 in 2023 and their educational qualifications."
prompt1 = prompt_question_tables.format(tables=text_tables, question=scenario1)

# Query GPT for scenario 1
response1 = query_gpt(prompt1)

# Print GPT response
print("Scenario 1 Response:", response1)

# Example Query (Version 2)
scenario2 = "Identify employees working in the 'Engineering' department who have attended Ivy League universities."
prompt2 = prompt_question_tables.format(tables=text_tables, question=scenario2)

# Query GPT for scenario 2
response2 = query_gpt(prompt2)

# Print GPT response
print("Scenario 2 Response:", response2)

Scenario 1 Response: To address the user's question, the following tables would be necessary to query with SQL:
1. employees
2. salary
3. studies
Scenario 2 Response: To address the user's question, the following tables would be necessary to query with SQL:
1. employees
2. studies

These tables will provide the necessary information to identify employees working in the 'Engineering' department who have attended Ivy League universities.


## Summary Report on GPT Prompting with SQL Queries

## Findings

Two different SQL-related scenarios were tested to evaluate how clearly and accurately GPT could identify the required database tables based on provided descriptions.

## Scenario 1
The query asked GPT to identify tables necessary for listing employees earning above $100,000 in 2023 and their educational qualifications. GPT correctly identified the tables:
- `employees`
- `salary`
- `studies`

The response was precise, accurate, and complete, aligning well with expectations.

## Scenario 2
The query requested GPT to identify employees from the 'Engineering' department who attended Ivy League universities. GPT correctly identified:
- `employees`
- `studies`

This scenario demonstrated GPT's strong contextual understanding and ability to filter relevant tables based on content descriptions.

## Variations and Issues
No significant hallucinations or incorrect identifications occurred in these specific test cases. Both prompts were clear enough that GPT responded accurately. However, GPT might occasionally misinterpret ambiguous queries, highlighting the importance of clear and explicit wording in prompts.

## Lessons Learned
- Clearly structured and detailed table descriptions significantly improve GPT's ability to provide accurate responses.
- GPT reliably identifies relevant tables if scenarios are straightforward and clearly described.
- Ambiguous queries or insufficient context might lead to inaccuracies; thus, precision in prompting is essential.

Overall, GPT effectively handled these scenarios, demonstrating strong potential as a supportive tool in database query planning and preliminary analysis.

